In [1]:
import matplotlib.pyplot as plt
from IPython.display import Image
import geemap
import numpy as np
import ee
import datetime

In [2]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [ ]:
wildfire_date = datetime.datetime.strptime('2021-07-13', '%Y-%m-%d')
wildfire_lat = 39.819
wildfire_lon = -121.419

In [4]:
image_elevation = ee.Image("NASA/NASADEM_HGT/001").select('elevation')
params_elevation = {'min': 0, 'max': 2000}

image_temperature = ee.ImageCollection("MODIS/061/MOD11A1").filterDate(ee.DateRange(wildfire_date, wildfire_date + datetime.timedelta(weeks=25))).select('LST_Day_1km')
def calculateWeeklyAverages(image_collection):
    weekly_averages = []
    for i in range(25):
        start_date = wildfire_date + datetime.timedelta(weeks=i)
        end_date = wildfire_date + datetime.timedelta(weeks=i+1)
        weekly_collection = image_collection.filterDate(start_date, end_date)
        weekly_mean = weekly_collection.mean()
        weekly_averages.append(weekly_mean)
    return ee.ImageCollection(weekly_averages)
image_temperature_weekly = calculateWeeklyAverages(image_temperature)
params_temperature = {'min': 13000, 'max': 16500,'palette': ['040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6', '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef', '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f', 'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d', 'ff0000', 'de0101', 'c21301', 'a71001', '911003']}

image_biomass = ee.ImageCollection("WCMC/biomass_carbon_density/v1_0")
params_biomass = {'min': 1, 'max': 180, 'palette': ['d9f0a3', 'addd8e', '78c679', '41ab5d', '238443', '005a32']}

In [ ]:
Map = geemap.Map()
Map.set_center(wildfire_lon, wildfire_lat, 6)

Map.add_layer(image_elevation, params_elevation, 'elevation')

Map.addLayer(image_temperature_weekly, params_temperature, 'Weekly Temperature Average')
Map.add_time_slider(image_temperature_weekly, params_temperature, labels=[(wildfire_date + datetime.timedelta(weeks=i)).strftime('%Y-%m-%d')+' to '+(wildfire_date + datetime.timedelta(weeks=i+1)).strftime('%Y-%m-%d') for i in range(25)])

Map.add_layer(image_biomass.select('carbon_tonnes_per_ha'), params_biomass, 'biomass')

Map